In [2]:
import pandas as pd
import numpy as np

In [53]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [75]:
weather_df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv")
solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
weather_df.sort_values(by='ref_datetime', inplace=True)
# weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
# weather_df.reset_index(inplace=True)

In [ ]:
weather_df["ref_datetime"] = pd.to_datetime(weather_df["ref_datetime"], utc=True)
weather_df["valid_datetime"] = pd.to_datetime(weather_df["valid_datetime"], utc=True)
weather_df["date_diff"] = (weather_df["valid_datetime"].dt.floor("D") - weather_df["ref_datetime"].dt.floor("D")).dt.days
weather_df = weather_df[(weather_df["ref_datetime"].dt.time == pd.to_datetime("06:00:00").time()) & (weather_df["date_diff"] == 1)]
weather_df = weather_df.sort_values('ref_datetime').groupby(['valid_datetime', 'latitude', 'longitude'], as_index=False).last()
weather_df.drop_duplicates(inplace=True)

In [76]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is depr

In [78]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
#df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled.drop(columns= ["index"], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]
df_resampled_merged_solar.ref_datetime = pd.to_datetime(df_resampled_merged_solar.ref_datetime)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/721348887.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar.ref_datetime = pd.to_datetime(df_resampled_merged_solar.ref_datetime)


In [79]:
df_resampled_merged_solar

,valid_datetime,latitude,longitude,ref_datetime,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp
0,2024-10-07 00:00:00+00:00,51.875051,0.911503,2024-10-07 00:00:00+00:00,14.420000,4.350000,8.530,226.900000,0.770,92.440000,99574.930000,0.00,NaN,2024-10-07 00:00:00+00:00,0.0,2956.745251,2779.542388
1,2024-10-07 00:30:00+00:00,51.875051,0.911503,NaT,14.270000,4.235000,8.255,231.445000,0.790,92.205000,99612.235000,0.00,NaN,2024-10-07 00:30:00+00:00,0.0,2956.745251,2779.542384
2,2024-10-07 01:00:00+00:00,51.875051,0.911503,2024-10-07 00:00:00+00:00,14.120000,4.120000,7.980,235.990000,0.810,91.970000,99649.540000,0.00,0.0,2024-10-07 01:00:00+00:00,0.0,2956.745251,2779.542383
3,2024-10-07 01:30:00+00:00,51.875051,0.911503,NaT,13.985000,4.295000,8.205,237.840000,0.815,91.730000,99649.540000,0.00,0.0,2024-10-07 01:30:00+00:00,0.0,2956.745251,2779.542370
4,2024-10-07 02:00:00+00:00,51.875051,0.911503,2024-10-07 00:00:00+00:00,13.850000,4.470000,8.430,239.690000,0.820,91.490000,99649.540000,0.00,0.0,2024-10-07 02:00:00+00:00,0.0,2956.745251,2779.542348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26715,2024-11-01 04:00:00+00:00,52.877668,1.191654,NaT,7.786667,1.453333,4.840,265.556667,0.190,96.096667,102644.186667,0.25,0.0,NaT,NaN,NaN,NaN
26716,2024-11-01 04:30:00+00:00,52.877668,1.191654,NaT,7.590000,1.410000,4.740,264.130000,0.240,96.515000,102643.150000,0.25,0.0,NaT,NaN,NaN,NaN
26717,2024-11-01 05:00:00+00:00,52.877668,1.191654,NaT,7.393333,1.366667,4.640,262.703333,0.290,96.933333,102642.113333,0.25,0.0,NaT,NaN,NaN,NaN
26718,2024-11-01 05:30:00+00:00,52.877668,1.191654,NaT,7.196667,1.323333,4.540,261.276667,0.340,97.351667,102641.076667,0.25,0.0,NaT,NaN,NaN,NaN


In [80]:
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [81]:
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [82]:
df_resampled_merged_solar1.head()

,valid_datetime,latitude,longitude,ref_datetime,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,14.12700,4.81800,8.3690,226.31250,0.89300,93.61300,99466.75350,0.0,NaN,2024-10-07 00:00:00+00:00,0.0,2956.745251,2779.542388
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,NaT,13.96225,4.93950,8.5185,232.54025,0.89825,92.78875,99514.00425,0.0,NaN,2024-10-07 00:30:00+00:00,0.0,2956.745251,2779.542384
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,13.79750,5.06100,8.6680,238.76800,0.90350,91.96450,99561.25500,0.0,0.0135,2024-10-07 01:00:00+00:00,0.0,2956.745251,2779.542383
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,NaT,13.53700,5.07225,8.7065,239.91850,0.86175,91.31550,99557.52450,0.0,0.0090,2024-10-07 01:30:00+00:00,0.0,2956.745251,2779.542370
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,13.27650,5.08350,8.7450,241.06900,0.82000,90.66650,99553.79400,0.0,0.0045,2024-10-07 02:00:00+00:00,0.0,2956.745251,2779.542348


In [83]:
def set_up_solar_features(df):
    df["cos_day"] = np.cos(2 * np.pi * df.valid_datetime.dt.dayofyear / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.valid_datetime.dt.hour / 24)
    df["SolarDownwardRadiation_Mean"] = df.SolarDownwardRadiation
    df["Temperature_dwd_Mean"] = df.Temperature
    df["Temperature_dwd_Std"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df.SolarDownwardRadiation_Mean.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1h"] = df.SolarDownwardRadiation_Mean.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.SolarDownwardRadiation_Mean.shift(1)
    df["SolarDownwardRadiation_Mean_Lag_1h"] = df.SolarDownwardRadiation_Mean.shift(2)
    df["SolarDownwardRadiation_Mean_Lag_24h"] = df.SolarDownwardRadiation_Mean.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [84]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [85]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])

In [86]:
df_resampled_merged_solar2["Panel_Temperature_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)

In [49]:
df_resampled_merged_solar2["Solar_MWh_Lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96) * 0.5
df_resampled_merged_solar2["Solar_MWh_credit"] = df_resampled_merged_solar2.generation_mw * 0.5
df_resampled_merged_solar2["Capacity_MWP_Lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)

In [87]:
df_resampled_merged_solar2

,valid_datetime,latitude,longitude,ref_datetime,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,...,Panel_Temperature_Point17,Panel_Efficiency_Point17,Panel_Temperature_Point18,Panel_Efficiency_Point18,Panel_Temperature_Point19,Panel_Efficiency_Point19,Panel_Temperature_Mean,Panel_Efficiency_Mean,Panel_Temperature_Std,Panel_Efficiency_Std
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,14.12700,4.81800,8.369000,226.312500,0.89300,93.61300,...,13.610000,0.188201,14.260000,0.187733,14.150000,0.187812,14.127000,0.187829,0.413825,0.000298
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,NaT,13.96225,4.93950,8.518500,232.540250,0.89825,92.78875,...,13.370000,0.188374,14.045000,0.187888,14.150000,0.187812,13.962250,0.187947,0.427970,0.000308
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,13.79750,5.06100,8.668000,238.768000,0.90350,91.96450,...,13.130000,0.188546,13.830000,0.188042,14.150000,0.187812,13.797500,0.188066,0.452911,0.000326
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,NaT,13.53700,5.07225,8.706500,239.918500,0.86175,91.31550,...,12.885000,0.188723,13.465000,0.188305,13.855000,0.188024,13.537000,0.188253,0.501444,0.000361
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,2024-10-07 00:00:00+00:00,13.27650,5.08350,8.745000,241.069000,0.82000,90.66650,...,12.640000,0.188899,13.100000,0.188568,13.560000,0.188237,13.276500,0.188441,0.580786,0.000418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2024-11-01 04:00:00+00:00,52.381681,0.650333,NaT,7.05800,1.50850,4.691667,258.709167,0.25950,96.88000,...,17.720156,0.185241,15.788750,0.186632,15.727031,0.186677,17.220906,0.185601,1.023251,0.000737
1209,2024-11-01 04:30:00+00:00,52.381681,0.650333,NaT,6.95025,1.48100,4.622500,258.085750,0.28325,97.04725,...,16.981875,0.185773,15.076875,0.187145,14.622187,0.187472,16.411500,0.186184,0.937882,0.000675
1210,2024-11-01 05:00:00+00:00,52.381681,0.650333,NaT,6.84250,1.45350,4.553333,257.462333,0.30700,97.21450,...,15.869375,0.186574,14.482031,0.187573,14.099063,0.187849,15.479367,0.186855,0.667206,0.000480
1211,2024-11-01 05:30:00+00:00,52.381681,0.650333,NaT,6.73475,1.42600,4.484167,256.838917,0.33075,97.38175,...,14.756875,0.187375,13.887188,0.188001,13.575938,0.188225,14.547234,0.187526,0.429520,0.000309


In [90]:
df_resampled_merged_solar2.columns.values

array(['valid_datetime', 'latitude', 'longitude', 'ref_datetime',
       'Temperature', 'WindSpeed', 'WindSpeed:100', 'WindDirection:100',
       'CloudCover', 'RelativeHumidity', 'PressureReducedMSL',
       'SolarDownwardRadiation', 'TotalPrecipitation', 'timestamp_utc',
       'generation_mw', 'installed_capacity_mwp', 'capacity_mwp',
       'cos_day', 'cos_hour', 'SolarDownwardRadiation_Mean',
       'Temperature_dwd_Mean', 'Temperature_dwd_Std',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Temperature_0',
       'SolarDownwardRadiation_0', 'Temperature_1',
       'SolarDownwardRadiation_1', 'Temperature_2',
       'SolarDownwardRadiation_2', 'Temperature_3',
       'SolarDownwardRadiation_3', 'Temperature_4',
       'SolarDownwardRadiation_4', 'Temperature_5',
       'SolarDownwardRadiation

## Changes 

In [88]:
# df_resampled_merged_solar2["reference_time"] = pd.to_datetime(df_resampled_merged_solar2["valid_datetime"], utc=True)
# df_resampled_merged_solar2["valid_time"] = pd.to_datetime(df_resampled_merged_solar2["valid_datetime"], utc=True)

# df_resampled_merged_solar2["date_diff"] = (df_resampled_merged_solar2["valid_time"].dt.floor("D") - df_resampled_merged_solar2["reference_time"].dt.floor("D")).dt.days
# df_resampled_merged_solar2 = df_resampled_merged_solar2[(df_resampled_merged_solar2["reference_time"].dt.time == pd.to_datetime("06:00:00").time()) & (df_resampled_merged_solar2["date_diff"] == 1)]

In [89]:
# df_resampled_merged_solar2

,valid_datetime,latitude,longitude,ref_datetime,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,...,Panel_Efficiency_Point18,Panel_Temperature_Point19,Panel_Efficiency_Point19,Panel_Temperature_Mean,Panel_Efficiency_Mean,Panel_Temperature_Std,Panel_Efficiency_Std,reference_time,valid_time,date_diff


In [15]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h",
    "Solar_MWh_credit"
    ]]
df_resampled_merged_solar3.dropna(inplace=True)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2103203449.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


In [16]:
df_resampled_merged_solar3.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Capacity_MWP_Lag_48h', 'Solar_MWh_Lag_48h',
       'Solar_MWh_credit'],
      dtype='object')

In [17]:
df_resampled_merged_solar3.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test.csv", index=False)

## Test normalize target

In [18]:
import numpy as np

df_resampled_merged_solar2["Target_Capacity_MWP_%"] = np.where(df_resampled_merged_solar2["Solar_MWh_credit"] == 0.0, 0, df_resampled_merged_solar2["Solar_MWh_credit"] / df_resampled_merged_solar2["capacity_mwp"])
df_resampled_merged_solar2["Target_Capacity_MWP_%_Lag_48"] = df_resampled_merged_solar2["Target_Capacity_MWP_%"].shift(96)

In [19]:
df_resampled_merged_solar2["capacity_mwp"].mean()

np.float64(2778.9164693780767)

In [25]:
df_resampled_merged_solar4 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Solar_MWh_Lag_48h",
    "Capacity_MWP_Lag_48h",
    "Target_Capacity_MWP_%_Lag_48",
    "Target_Capacity_MWP_%",
    ]]
df_resampled_merged_solar4.dropna(inplace=True)

KeyError: "['valid_time', 'reference_time'] not in index"

In [21]:
df_resampled_merged_solar4

,SolarDownwardRadiation_Mean,SolarDownwardRadiation_RW_Mean_1h,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_Mean_Lag_1h,SolarDownwardRadiation_Mean_Lag_24h,Panel_Efficiency_Mean,Panel_Efficiency_Std,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,cos_hour,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h,Target_Capacity_MWP_%_Lag_48,Target_Capacity_MWP_%
96,0.0,0.0,0.0,0.0,0.0,0.0,0.188081,0.000357,13.77600,0.496404,0.337606,13.58750,1.000000,0.158559,0.0,2779.542388,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.188098,0.000327,13.75300,0.454666,0.349814,13.48025,1.000000,0.158559,0.0,2779.542384,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.188098,0.000327,13.75300,0.454666,0.370293,13.37300,0.965926,0.158559,0.0,2779.542383,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.188134,0.000287,13.70275,0.399291,0.364783,13.32325,0.965926,0.158559,0.0,2779.542370,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.188134,0.000287,13.70275,0.399291,0.362641,13.27350,0.866025,0.158559,0.0,2779.542348,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,0.0,0.0,0.0,0.0,0.0,0.0,0.189700,0.000483,11.52750,0.671118,0.377214,8.80200,0.707107,0.452072,0.0,2778.376631,0.0,0.0
967,0.0,0.0,0.0,0.0,0.0,0.0,0.189715,0.000476,11.50700,0.661079,0.304786,8.74125,0.707107,0.452072,0.0,2778.376610,0.0,0.0
968,0.0,0.0,0.0,0.0,0.0,0.0,0.189730,0.000475,11.48650,0.660010,0.291123,8.68050,0.500000,0.452072,0.0,2778.376599,0.0,0.0
969,0.0,0.0,0.0,0.0,0.0,0.0,0.189765,0.000458,11.43800,0.635788,0.281312,8.69450,0.500000,0.452072,0.0,2778.376598,0.0,0.0


In [22]:
df_api = df_resampled_merged_solar4.copy()

In [24]:
# df_api["reference_time"] = pd.to_datetime(df_api["valid_datetime"], utc=True)
# df_api["valid_time"] = pd.to_datetime(df_api["valid_datetime"], utc=True)

# df_api["date_diff"] = (df_api["valid_time"].dt.floor("D") - df_api["reference_time"].dt.floor("D")).dt.days
# df_api = df_api[(df_api["reference_time"].dt.time == pd.to_datetime("06:00:00").time()) & (df_api["date_diff"] == 1)]

KeyError: 'valid_datetime'

In [23]:
df_resampled_merged_solar4.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test_norm.csv", index=False)